In [1]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
import datetime
import csv
import datetime
import pandas as pd
from django.core.mail import send_mail

#  smtplib - SMTP client session object that can be used to send mail to any internet machine
import smtplib

In [2]:
# Connecting to the website and pulling in data

URL = 'https://www.amazon.in/OnePlus-Nord-Blue-128GB-Storage/dp/B097RD2JX8/ref=sr_1_2?crid=1OR2F8ATZBCKD&keywords=nord+2&qid=1646402885&s=electronics&sprefix=nord%2Celectronics%2C433&sr=1-2'

# url to get my user agent - https://httpbin.org/get 
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
           (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36", "X-Amzn-Trace-Id": "Root=1-631eb784-2bc5c7815da69dad2823dbab"}

page = requests.get(URL, headers = headers)

#pulling in all the html
soup1 = BeautifulSoup(page.content, 'html.parser')

soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

#getting the title of the product from the page
title = soup2.find(id = 'productTitle').get_text()

#getting the price of the product from the page
#https://stackoverflow.com/questions/34301815/understand-the-find-function-in-beautiful-soup
price = soup2.find("span", {"class": "a-offscreen"}).get_text()

print(title)
print(price)


            OnePlus Nord 2 5G (Blue Haze, 8GB RAM, 128GB Storage)
           

                       ₹27,899.00
                      


In [3]:
#removing the unnecessary spaces
title = title.strip()
price = price.strip()

#removing comma and rupees symbol from the price string
price = price.replace(',', '')
price = price.replace('₹' , '')

#Converting the price string to int so that it can be compared later on
price = int(float(price))

print(title)
print(price)

OnePlus Nord 2 5G (Blue Haze, 8GB RAM, 128GB Storage)
27899


In [4]:
#get current date
today = datetime.date.today()
print(today)

2022-09-12


In [6]:
header = ['Title', 'Price', 'Date']
data = [title, price, today]

#creating the csv
with open('Amazon_Web_Scraper_Dataset.csv', 'w', newline = '', encoding = 'UTF8') as f:
    writer = csv.writer(f)
    
    #inserting header
    writer.writerow(header)
    
    #inserting the data
    writer.writerow(data)
    

In [7]:
df=pd.read_csv(r'C:\Users\rr159\Amazon_Web_Scraper_Dataset.csv')
df

,Title,Price,Date
0,"OnePlus Nord 2 5G (Blue Haze, 8GB RAM, 128GB S...",27899,2022-09-12


In [8]:
#appending the data 

with open('Amazon_Web_Scraper_Dataset.csv', 'a+', newline = '', encoding = 'UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)
    

In [9]:
df=pd.read_csv(r'C:\Users\rr159\Amazon_Web_Scraper_Dataset.csv')
df

,Title,Price,Date
0,"OnePlus Nord 2 5G (Blue Haze, 8GB RAM, 128GB S...",27899,2022-09-12
1,"OnePlus Nord 2 5G (Blue Haze, 8GB RAM, 128GB S...",27899,2022-09-12


In [10]:
#defining a function to check price, append the data in csv and if price of the smartphone below Rs.26000 send an email

def check_price():
    URL = 'https://www.amazon.in/OnePlus-Nord-Blue-128GB-Storage/dp/B097RD2JX8/ref=sr_1_2?crid=1OR2F8ATZBCKD&keywords=nord+2&qid=1646402885&s=electronics&sprefix=nord%2Celectronics%2C433&sr=1-2'

    # url to get my user agent - https://httpbin.org/get 
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
           (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36", "X-Amzn-Trace-Id": "Root=1-62221d56-546b2fdf5ba0259122dab2aa"}

    page = requests.get(URL, headers = headers)

    #pulling in all the html
    soup1 = BeautifulSoup(page.content, 'html.parser')

    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    #getting the title of the product from the page
    title = soup2.find(id = 'productTitle').get_text()

    #getting the price of the product from the page
    #https://stackoverflow.com/questions/34301815/understand-the-find-function-in-beautiful-soup
    price = soup2.find("span", {"class": "a-offscreen"}).get_text()
       
    #removing the unnecessary spaces
    title = title.strip()
    price = price.strip()

    #removing comma and rupees symbol from the price string
    price = price.replace(',', '')
    price = price.replace('₹' , '')

    #Converting the price string to int so that it can be compared later on
    price = int(float(price))

    #get current date
    today = datetime.date.today()
    
    #defining the header and data for the csv file
    header = ['Title', 'Price', 'Date']
    data = [title, price, today]
    
    #appending the data in the csv file
    with open('Amazon_Web_Scraper_Dataset.csv', 'a+', newline = '', encoding = 'UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
        
    #To send the email alert when price of the phone is below Rs.26000    
    if(price < 26000):
        send_email()

In [ ]:
#automated it to get the price of the product and append the data every 24 hours in csv file , 
#and if price less than Rs.26000 send an email

while(True):
    check_price()
    time.sleep(86400) #1 day = 86400 seconds

In [ ]:
#To send the email alert

#https://realpython.com/python-send-email/
def send_email():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()  #Identify yourself to an ESMTP server using ehlo()
    #server.starttls()
    server.ehlo()
    server.login('xxxxxxxx','xxxxxx') #need to give mail id and password
    
    subject = "OnePlus Nord 2 is below Rs.26000! Time to buy!"
    body = "Adhip, Check the Sale. \
    Now is the time to buy the mobile phone at discounted price. \
    Link here: https://www.amazon.in/OnePlus-Nord-Blue-128GB-Storage/dp/B097RD2JX8/ref=sr_1_2?crid=1OR2F8ATZBCKD&keywords=nord+2&qid=1646402885&s=electronics&sprefix=nord%2Celectronics%2C433&sr=1-2"
   
    message = f"Subject: {subject}\n\n{body}"  
    
    server.sendmail('xxxxxxxx','xxxxxxxx', message ) #need to give from(Sender) and to(Receiver) email address
    